### Собственно что это?

Как говорится не сломаешь не поймешь: используя знания от безусловной генерации я хочу попробовать придумать условную генерацию по метке

In [12]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

import pandas as sns
import numpy as np
import matplotlib.pyplot as plt

import kagglehub
import idx2numpy
import os

In [13]:
def read_mnist():
    path = kagglehub.dataset_download("hojjatk/mnist-dataset")
    print("Путь до данных:", path)

    train_images_path = os.path.join(path, 'train-images.idx3-ubyte')
    train_labels_path = os.path.join(path, 'train-labels.idx1-ubyte')
    test_imaages_path = os.path.join(path, 't10k-images.idx3-ubyte')
    test_labels_path = os.path.join(path, 't10k-labels.idx1-ubyte')
    
    X_train = idx2numpy.convert_from_file(train_images_path)
    y_train = idx2numpy.convert_from_file(train_labels_path)
    X_test = idx2numpy.convert_from_file(test_imaages_path)
    y_test = idx2numpy.convert_from_file(test_labels_path)

    print(f'Загружено тренировочных картинток: {len(X_train)}')
    print(f'Загружено тренировочных меток: {len(y_train)}')
    print(f'Загружено тестовых картинток: {len(X_test)}')
    print(f'Загружено тестовых меток: {len(y_test)}')
    
    return X_train, X_test, y_train, y_test

In [14]:
X_train, X_test, y_train, y_test = read_mnist()

Путь до данных: C:\Users\panika\.cache\kagglehub\datasets\hojjatk\mnist-dataset\versions\1
Загружено тренировочных картинток: 60000
Загружено тренировочных меток: 60000
Загружено тестовых картинток: 10000
Загружено тестовых меток: 10000


In [16]:
def normalize_data(X):
    X_zero_one = X / 255.0
    X_minus_one_one = (X_zero_one - 0.5) * 2.0
    return X_minus_one_one

batch_size = 128
X = torch.tensor(X_train, dtype=torch.float32)
X_norm = normalize_data(X)
X_batch = X_norm.view(-1, 1, 28, 28)
train_loader = DataLoader(TensorDataset(X_batch), batch_size=batch_size, shuffle=True)

### Собственно какие идеи у меня?

Мне очень нравится идеи с тем, что аддитивно добавляя аддитивно время картинка перемещается в свою область. Очевидно, я буду отображать метку в какой то вектор, а после добавлять также аддитивно (хотя можно и мультипликативно), но есть мысль, что если два раза пройтись ReLu(Conv(Relu(Conv(x)) + t)), то может быть проблема того, что информация просто потеряется и хотелось бы научиться это делать параллельно

In [ ]:
def Block(nn.Module):
    pass

def Unet(nn.Module):
    pass